In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import re
import pandas as pd 

In [3]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("광진구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
try:
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
    )
    driver.switch_to.frame(iframe)
except Exception as e:
    print(f"iframe 전환 오류: {e}")
    driver.quit()
    exit()

# 결과 리스트 초기화
results = []
visited_urls = set()
page_number = 1
MAX_PAGES = 5  # 최대 페이지 수
MAX_RESULTS = 150  # 최대 결과 수 제한

while page_number <= MAX_PAGES and len(results) < MAX_RESULTS:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store in store_elements:
            if len(results) >= MAX_RESULTS:
                print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
                break

            try:
                store.click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")

        if len(results) >= MAX_RESULTS:
            print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
            break

        # '다음' 버튼 클릭
        if page_number < MAX_PAGES:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="eUTV2" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
                time.sleep(random.uniform(5, 10))

                # iframe 전환
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
                break

        # 페이지 번호 증가
        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

# CSV 파일 저장
file_name = "Gwangjin-gu_restaurants.csv"
df = pd.DataFrame(results)
df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 탄토탄토
2. 정통숯불꼬치구이구르메 군자점
3. 백일도
4. 홍별감네
5. 건대사시미
6. 마초쉐프 건대점
7. 해피니스디저트
8. 코후쿠
9. 남자수산
10. 박과장의 조은회사
11. 만경상회 건대본점
12. 오마카세 오사이초밥 건대본점
13. 송화산시도삭면
14. 고향양꼬치 건대점
15. 킨보이서울
16. 회식의달인 건대점
17. 에이뿔램 군자점
18. 연길왕꼬치 건대본점
19. 멕시칼리
20. 매운향솥 마라탕 마라샹궈
21. 로니로티 건대점
22. 최원석의돼지한판&서해쭈꾸미 건대1호점
23. 바인
24. 시치린
25. 비스타 워커힐 서울 더뷔페
26. 허씨 정통중경마라샹궈 3호점
27. 이이요
28. 미도인 건대
29. 레스또 커피
30. 비스타 워커힐 서울 피자힐
31. 도치피자 강변
32. 예담밥상
33. 민지슈퍼 2호점
34. 뽕사부
35. 수작나베 강변본점
36. 하이디라오 건대지점
37. 신선술집 도원
38. 행복한우
39. 한남성왕꼬치 건대본점
40. 더 클래식 500 라구뜨
41. 찌마기 구의점
42. 윤숲 후르츠산도점
43. 이화만두
44. 호야초밥참치 본점
45. 스윗어스슈
46. 숙성회136 군자점
47. 육일관 건대본점
48. 건대 고기굽는놈 본점
49. 구의규 구의본점
50. 늘조은한우 구의직영점
51. 최원석의돼지한판&화식한우광장점
52. 고육담 건대본점
53. 솔방울
54. 요비
55. 굴이짱
56. 해란양꼬치 건대본점
57. 바다회포차
58. 건대포항막회
59. 코시 건대점
60. 구구향훠궈 건대직영점
61. 노릇
62. 재즈라운지
63. 스위트앤카츠
64. 송화산시도삭면 2호점
65. 장미술상 건대점
66. 연옥양꼬치
67. daughter
68. 민지슈퍼 건대본점
69. 니뽕내뽕 건대본점
70. 간장삼겹살 동방생고기 건대점
71. 릴리베이커리
72. 신토불이 떡볶이 본점
73. 순곱이네 건대점
74. 브런치 카페 고은별 샌드위치
75. 육회한연어 건대점
76. 산재우
'다